In [14]:
# Packages and libraries
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


import matplotlib.pyplot as plt

from matplotlib import pyplot as plt


#################### KØR KUN PLAIN MED DENNE
def func_xg(df, model_name):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds'])
    mse_resultater=[]
    for i in range(200):
        df = df.copy()

        if model_name == 'WO':
            X = df.drop(['COL_GRADE_AVG','Unnamed: 0','CR_S11','CC_S11','ENG_S11','CR_PRO','CC_PRO','ENG_PRO', 'STRATUM','GENDER','SCHOOL_NAT',	'SCHOOL_TYPE',	'MAT_S11'	,'BIO_S11'], axis=1).copy()
            y=df['COL_GRADE_AVG'].copy()
            X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=i, test_size=0.33)#, stratify=y)
            for_later_gender = X_test.drop(['HI_GRADE_AVG','QR_PRO', 'WC_PRO'], axis = 1)
            X_train = X_train.drop('GENDER_bin', axis = 1)
            X_test = X_test.drop('GENDER_bin', axis = 1)
            gender = for_later_gender.to_numpy()
        
        else:
            X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_PRO','CC_PRO','ENG_PRO'], axis=1).copy()
            X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_PRO','CC_PRO','ENG_PRO','BIO_S11','MAT_S11','QR_PRO','WC_PRO'], axis=1).copy()                
                
            y=df['COL_GRADE_AVG'].copy()
            # for_later_gender = X_test.drop(['HI_GRADE_AVG','QR_PRO',	'WC_PRO'], axis = 1)
            
            X_encoded = pd.get_dummies(X,columns=['STRATUM','SCHOOL_TYPE','SCHOOL_NAT'])
            X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,random_state=i, test_size=0.33)#, stratify=y)
            gender = X_test.GENDER_bin.to_numpy()

            # print(X_test)
      
        dfs = X_train.copy()
        for column in dfs.columns:
            try:
                dfs[column] = (dfs[column] - dfs[column].mean())/dfs[column].std()  
            except:
                pass
        X_train = dfs
        
        dft = X_test.copy()
        for column in dft.columns:
            try:
                dft[column] = (dft[column] - dft[column].mean())/dft[column].std()  
            except:
                pass

        X_test = dft

        final_model = xgb.XGBRegressor(seed=24, objective='reg:squarederror', max_depth=3, learning_rate=0.05, gamma=0,reg_lambda=10.0, scale_pos_weight=1)

        final_model.fit(X_train,y_train,verbose=True, eval_metric='rmse')

        y_pred_final = final_model.predict(X_test)
        mse_resultater.append(mean_squared_error(y_test, y_pred_final))
        
        index = X_test.index.to_numpy()
    
        preds = y_pred_final
        listen = list(map(list, zip(index,gender, preds, y_test)))

        samlet_preds_index_without_sensitive = pd.DataFrame(listen, columns=['Idx','Gender_bins','Preds','y_test'])
        samlet_preds_index_without_sensitive = samlet_preds_index_without_sensitive.sort_values(by=['Preds'],ascending=False)

        samlet_preds_index_without_sensitive = samlet_preds_index_without_sensitive.reset_index()     


        tom_res = tom_res.append(samlet_preds_index_without_sensitive)

    tom_res = tom_res.drop(['index','not_4_use'], axis=1)
    # return tom_res.to_csv(f"func_xg_wo_res/mse_resultater.csv")
    return mse_resultater
    # return tom_res.to_csv(f"Bachelor_Gabi_Kat/fairsearch-fair-python/samlet_preds_index_ny_{model_name}.csv")

In [17]:
df = pd.read_csv("GUDF.csv")


np.mean(func_xg(df, model_name='PLAIN'))

208.72484175383607

In [18]:
# Packages and libraries
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


import matplotlib.pyplot as plt

from matplotlib import pyplot as plt


def func_xg_WO(df, model_name):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds'])
    mse_resultater=[]
    for i in range(200):
        df = df.copy()

        if model_name == 'WO':
            X = df.drop(['COL_GRADE_AVG','Unnamed: 0','CR_S11','CC_S11','ENG_S11','CR_PRO','CC_PRO','ENG_PRO', 'STRATUM','GENDER','SCHOOL_NAT',	'SCHOOL_TYPE',	'MAT_S11'	,'BIO_S11','QR_PRO','WC_PRO'], axis=1).copy()
            y=df['COL_GRADE_AVG'].copy()
            X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=i, test_size=0.33)#, stratify=y)
            for_later_gender = X_test.drop(['HI_GRADE_AVG'], axis = 1) #['HI_GRADE_AVG','QR_PRO', 'WC_PRO'
            X_train = X_train.drop('GENDER_bin', axis = 1)
            X_test = X_test.drop('GENDER_bin', axis = 1)
            gender = for_later_gender.to_numpy()
            
        
        else:
            X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_PRO','CC_PRO','ENG_PRO','BIO_S11','ENG_S11','QR_PRO','WC_PRO'], axis=1).copy()                
               
            y=df['COL_GRADE_AVG'].copy()
            # for_later_gender = X_test.drop(['HI_GRADE_AVG','QR_PRO',	'WC_PRO'], axis = 1)
            
            X_encoded = pd.get_dummies(X,columns=['STRATUM','SCHOOL_TYPE','SCHOOL_NAT'])
            X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,random_state=i, test_size=0.33)#, stratify=y)
            gender = X_test.GENDER_bin.to_numpy()
      
        dfs = X_train.copy()
        for column in dfs.columns:
            try:
                dfs[column] = (dfs[column] - dfs[column].mean())/dfs[column].std()  
            except:
                pass
        X_train = dfs
        
        dft = X_test.copy()
        for column in dft.columns:
            try:
                dft[column] = (dft[column] - dft[column].mean())/dft[column].std()  
            except:
                pass

        X_test = dft

        final_model = xgb.XGBRegressor(seed=24, objective='reg:squarederror', max_depth=3, learning_rate=0.05, gamma=0,reg_lambda=15.0, scale_pos_weight=0.01)
        # final_model = xgb.XGBRegressor(seed=24, objective='reg:squarederror', max_depth=3, learning_rate=0.05, gamma=0,reg_lambda=10.0, scale_pos_weight=1)

        final_model.fit(X_train,y_train,verbose=True, eval_metric='rmse')

        y_pred_final = final_model.predict(X_test)
        mse_resultater.append(mean_squared_error(y_test, y_pred_final))

        
        index = X_test.index.to_numpy()
    
        preds = y_pred_final
        listen = list(map(list, zip(index,gender, preds, y_test)))

        samlet_preds_index_without_sensitive = pd.DataFrame(listen, columns=['Idx','Gender_bins','Preds','y_test'])
        samlet_preds_index_without_sensitive = samlet_preds_index_without_sensitive.sort_values(by=['Preds'],ascending=False)

        samlet_preds_index_without_sensitive = samlet_preds_index_without_sensitive.reset_index()
        
        # samlet_preds_index_without_sensitive['Gender_bins'] = samlet_preds_index_without_sensitive['Gender_bins'].str.strip('[]').astype(int)
        


        # result = samlet_preds_index_without_sensitive.to_csv(f"func_xg_wo_res/samlet_preds_index_without_sensitive_ny{i}_{model_name}.csv")
        tom_res = tom_res.append(samlet_preds_index_without_sensitive)

    tom_res = tom_res.drop(['index','not_4_use'], axis=1)
    # return tom_res.to_csv(f"func_xg_wo_res/RMSE_{model_name}.csv")
    return mse_resultater
    # return tom_res.to_csv(f"Bachelor_Gabi_Kat/fairsearch-fair-python/samlet_preds_index_ny_{model_name}.csv")

In [19]:
df = pd.read_csv("GUDF.csv")


np.mean(func_xg_WO(df, model_name='WO'))

209.2500072431888

In [28]:
def data_splitter(df):
    tom_res = pd.DataFrame()#columns=['MAT_S11','CR_S11','CC_S11','HI_GRADE_AVG','GENDER_bin','STRATUM_0','STRATUM_Stratum_1','STRATUM_Stratum_2','STRATUM_Stratum_3','STRATUM_Stratum_4','STRATUM_Stratum_5','STRATUM_Stratum_6','SCHOOL_TYPE_ACADEMIC','SCHOOL_TYPE_Not_apply','SCHOOL_TYPE_TECHNICAL','SCHOOL_TYPE_TECHNICAL/ACADEMIC','SCHOOL_NAT_PRIVATE','SCHOOL_NAT_PUBLIC','COL_GRADE_AVG'])
    for i in range(200):
        df = df.copy()
        X = df.drop(['COL_GRADE_AVG','GENDER','Unnamed: 0','CR_PRO','CC_PRO','ENG_PRO','BIO_S11','ENG_S11','QR_PRO','WC_PRO'], axis=1).copy()                
        y=df['COL_GRADE_AVG'].copy()

        X_encoded = pd.get_dummies(X,columns=['STRATUM','SCHOOL_TYPE','SCHOOL_NAT'])
        X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,random_state=i, test_size=0.33)#, stratify=y)
        gender = X_test.GENDER_bin.to_numpy()
        

        # X_test = dfs.sort_values(by=['Preds'],ascending=False)

        # dfs = dfs.reset_index()
        # result = pd.concat([df1, s1], axis=1, ignore_index=True)
     
        dfs = pd.concat([X_test, y_test], axis=1, ignore_index=True)# Her sættes en X_test og en y_test sammen
        


        tom_res = tom_res.append(dfs)
    #     print(tom_res)
    # print(tom_res)
    # tom_res = tom_res.drop(['index','not_4_use'], axis=1)
    return tom_res.to_csv(f"func_xg_wo_res/dfs_test{i}.csv")

In [29]:
# Packages and libraries
import pandas as pd
import xgboost as xgb
import numpy as np


from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold


import matplotlib.pyplot as plt

from matplotlib import pyplot as plt

import pandas as pd
df = pd.read_csv("GUDF.csv")
data_splitter(df)